## Implementation using TensorFlow on fashion minst dataset.

In [ ]:
from fashionmnist_model import FMM
import tensorflow as tf

In [ ]:
# Load the data
X_train, y_train, X_test, y_test = FMM.load_data()

In [ ]:
# Create and compile the model with the optimizer
optimizer = tf.keras.optimizers.Adam(
    **{
        "learning_rate": 0.001048353562375999,
        "beta_1": 0.9579452319747783,
        "beta_2": 0.970555575375257,
        "epsilon": 6.27029953769078e-08,
        "weight_decay": 3.464892852166533e-06,
    }
)
model = FMM.create_model()
print(f"Training with {optimizer.__class__.__name__} optimizer...")
history, train_accuracy, val_accuracy = FMM.compile_and_train(
    model, X_train, y_train, X_test, y_test, optimizer
)

In [ ]:
loss, accuracy = FMM.evaluate(model, X_test, y_test)

In [ ]:
print(f"Training accuracy : {train_accuracy}")
print(f"Validation accuracy : {val_accuracy}")
print(f"Loss : {loss}")
print(f"Accuracy : {accuracy}")

In [ ]:
FMM.plot_history(history, optimizer)

In [ ]:
raise KeyboardInterrupt

In [ ]:
! pip install -U "ray[data,train,tune,serve]"

In [ ]:
import ray
from ray import tune, train
from ray.tune import grid_search
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
ray.init()

In [ ]:
# Define a function to train the model
def train_model(config):
    # Load and preprocess the Fashion MNIST dataset
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train, X_test = X_train / 255.0, X_test / 255.0  # Normalize pixel values

    # Define a TensorFlow model
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

    # Compile the model
    optimizer = tf.keras.optimizers.legacy.Adam(
        learning_rate=config["learning_rate"],
        #         beta_1=config["beta_1"],
        #         beta_2=config["beta_2"],
        #         epsilon=config["epsilon"],
    )
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    # Train the model
    model.fit(X_train, y_train, epochs=50, verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    train.report({"accuracy": accuracy, "loss": loss})

In [ ]:
search_space = {
    "learning_rate": tune.loguniform(1e-4, 1e-2),  # Learning rate
    #     "beta_1": tune.uniform(0.8, 0.999),  # Beta1
    #     "beta_2": tune.uniform(0.8, 0.999),  # Beta2
    #     "epsilon": tune.loguniform(1e-10, 1e-6),  # Epsilon
    #     "weight_decay": tune.loguniform(1e-6, 1e-3),
}

In [ ]:
# Run hyperparameter tuning
analysis = tune.run(
    train_model,
    config=search_space,
    num_samples=16,
    metric="accuracy",
    mode="max",
    resources_per_trial={"cpu": 2},
    verbose=1,
)

In [ ]:
# Print the best hyperparameters and results
config = analysis.best_config
print("Best hyperparameters:", analysis.best_config)
print("Best accuracy:", analysis.best_result["accuracy"])

In [ ]:
optimizer = tf.keras.optimizers.Adam(**config)
model = FMM.create_model()
print(f"Training with {optimizer.__class__.__name__} optimizer...")
print(f"OPtimizer hyperparameters : {optimizer.get_config()}.")
history, train_accuracy, val_accuracy = FMM.compile_and_train(
    model, X_train, y_train, X_test, y_test, optimizer
)

In [ ]:
loss, accuracy = FMM.evaluate(model, X_test, y_test)

In [ ]:
print(f"Training accuracy : {train_accuracy}")
print(f"Validation accuracy : {val_accuracy}")
print(f"Loss : {loss}")
print(f"Accuracy : {accuracy}")

In [ ]:
FMM.plot_history(history, optimizer)

In [ ]:
ray.shutdown()